# LU-разложение

## Постановка задачи

Пусть дана квадратная матрица $A \in \mathbb{R}^{n \times n}$.
LU-разложение состоит в представлении матрицы $A$ в виде произведения двух треугольных матриц:

$$
A = L U
$$

где

- $L$ — нижняя треугольная матрица с единицами на диагонали
- $U$ — верхняя треугольная матрица

LU-разложение позволяет эффективно решать системы линейных уравнений $A x = b$ без вычисления обратной матрицы.




## Алгоритм построения LU-разложения

Пусть $A$ — матрица размера $n \times n$.
Инициализируем:

$$
L = I_n, \quad U = A
$$

где $I_n$ — единичная матрица.

### Шаги метода

Для каждого ведущего элемента $U[i,i]$, $i = 0,1,\dots,n-1$:

1. Проверяем, что элемент не слишком мал:

$$
|U[i,i]| < \varepsilon
$$

Если элемент слишком мал, ищем строку с максимальным по модулю элементом в столбце и меняем строки (частичный выбор ведущего).

2. Для каждой строки $j > i$ вычисляем множитель:

$$
L[j,i] = \frac{U[j,i]}{U[i,i]}.
$$

3. Вычитаем из строки $j$ строку $i$, умноженную на множитель:

$$
U[j,i:] = U[j,i:] - L[j,i] \cdot U[i,i:].
$$

После выполнения всех шагов получаем:

$$
A = L U
$$

## Решение системы $A x = b$ с помощью LU-разложения

1. Решаем прямую систему:

$$
L y = b
$$

методом прямого хода.

2. Решаем обратную систему:

$$
U x = y
$$

методом обратного хода.

В результате получаем решение исходной системы $x$.



In [102]:
import sympy
import numpy as np

In [103]:
A = np.array([[1, 2, 3, -2],
              [2, -1, -2, -3],
              [3, 2, -1, 2],
              [2, -3, 2, 1]], dtype=np.float64)
# вектор правой части системы Ax = b
b = np.array([1, 2, -5, 11], dtype=np.float64)

In [104]:
count_el_diag = min(A.shape)

In [105]:
L = np.zeros(A.shape, dtype=np.float64)
U = A.copy()

In [106]:
H = A.shape[0]
W = A.shape[1]
eps = 1e-12

for i in range(count_el_diag):
    L[i, i] = 1.0  # на главной диагонали L всегда стоят единицы

    # если текущий ведущий элемент равен нулю или очень маленький, меняем строки
    if abs(U[i, i]) < eps:
        max_row = np.argmax(np.abs(U[i:, i])) + i  # ищем максимум по модулю в столбце
        if abs(U[max_row, i]) < eps:
            continue

        if max_row != i:
            # меняем строки в U
            U[[i, max_row], :] = U[[max_row, i], :]
            # и уже посчитанную часть L
            if i > 0:
                L[[i, max_row], :i] = L[[max_row, i], :i]
            # меняем строки в векторе правой части b тоже, чтобы система оставалась корректной
            b[[i, max_row]] = b[[max_row, i]]

    # зануление элементов под диагональю
    for j in range(i + 1, H):
        mult = U[j, i] / U[i, i]  # коэффициент для зануления элемента

        L[j, i] = mult  # сохраняем множитель в L

        # вычитаем строку с множителем
        U[j, i:] = U[j, i:] - mult * U[i, i:]

        U[j, i] = 0.0  # зануляем элемент точно
        U[j][np.abs(U[j]) < eps] = 0.0  # убираем мелкий мусор

print("L =")
print(L)

print("\nU =")
print(U)

L =
[[ 1.   0.   0.   0. ]
 [ 2.   1.   0.   0. ]
 [ 3.   0.8  1.   0. ]
 [ 2.   1.4 -2.   1. ]]

U =
[[ 1.   2.   3.  -2. ]
 [ 0.  -5.  -8.   1. ]
 [ 0.   0.  -3.6  7.2]
 [ 0.   0.   0.  18. ]]


### Решение системы Ax = b
A = L * U  ->  L * U * x = b \
Сначала решаем: L * y = b (прямой ход) \
Потом решаем: U * x = y (обратный ход)


### Прямой ход

In [107]:
y = np.zeros(H)

for i in range(H):
    s = 0.0
    for j in range(i):
        s += L[i, j] * y[j]

    y[i] = b[i] - s  # прямой ход L y = b

print("\ny =")
print(y)


y =
[ 1.  0. -8. -7.]


### обратный ход


In [108]:
x = np.zeros(H)

for i in range(H - 1, -1, -1):
    s = 0.0
    for j in range(i + 1, H):
        s += U[i, j] * x[j]

    x[i] = (y[i] - s) / U[i, i]  # обратный ход U x = y

print("\nx =")
print(x)


x =
[ 0.66666667 -2.38888889  1.44444444 -0.38888889]
